<div id="header" align="center">

# Introducción

</div>

Para este proyecto se va a analizar una base de datos de usuarios de la compañía **Megaline**, los cuales están divididos en dos tipos de planes *Smart* y *Ultra*. 

El objetivo de éste analisis es el de crear un modelo de ML que pueda clasificar a los clientes basado en sus consumos de llamadas, mensajes y megabytes para así decidir si cada usuario debería pertener al plan *Smart* o si cambiar al plan *Ultra* sería una mejor opción.


### 1. Importación de librerias.

Debido a que se van a utilizar distintos modelos voy a importar las librerias y métodos vistos en el Sprint.

In [3]:
import pandas as pd
# Modelos de ML
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
# Segmentación de datos
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# Métricas de evaluación.
from sklearn.metrics import accuracy_score

### 2. Cargar los datos.

In [4]:
df = pd.read_csv('C:/datasets/Sprint 9/users_behavior.csv')

display(df.head())
print()
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### 3. Segmentación de los datos.

Voy a segmentar los datos en 3 con los siguientes porcentajes:

|    Conjunto   | Porcentaje |
|---------------|------------|
| Entrenamiento |    60 %    |
|   Validación  |    20 %    |
|    Pruebas    |    20 %    |

In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

# Primero separo en un 80% (entrenamiento + validación) y 20% (pruebas)  
x_train, features_test, y_train, target_test = train_test_split(features, target, test_size=0.2, random_state=123)

# Después el 80% se divide en el 25% y 75%. De este modo ((0.25 * 0.8) + (0.75 * 0.8)) + (0.2) = 1
features_train, features_val, target_train, target_val = train_test_split(x_train, y_train, test_size=0.25, random_state=123)

Utilicé la divisón train_test_split() dos veces, debido a que se van a separar los datos de la siguiente manera para evitar el sobreajuste con el modelo entrenado...

| Tipos de datos |% del total | sub % |
|----------------|------------|-------|
|  Entrenamiento |    60 %    |  75%  |
|    Validación  |    20 %    |  25%  |
|     Prueba     |    20 %    |       |

### 4. Prueba de diferentes modelos de ML.

Al buscar un modelo de decisión donde el resultado para las predicciones solo existen dos posibles respuestas (0 o 1). Los métodos más apropiados son los de Arboles de Decisión, Bosque Aleatorio, Regresióm Lógistca, SVM, KNN, Gradient Boosting y Naive Bayes.

#### 4.1 Arboles de Decisión.

Primero voy a entrenar el dataset de *features_train* con el algoritmo de aprendizaje *DecisionTreeClassifier* para obtener un modelo entrenado y después utilizar el *features_val* para validar las predicciones y ya después vealuar con diferentes métricas la calidad del modelo.

In [6]:
# Entrenamiento.
model_tree = DecisionTreeClassifier(random_state=123)
model_tree.fit(features_train,target_train)

# Aplicación.
pred_val = model_tree.predict(features_val)

##### 4.1.1 Métricas de evaluación.

Vamos a determinar la exactitud del modelo entrenado que generamos, comparando la variable *pred_val* contra *target_val*.

In [7]:
print(f'La exactitud del árbol de decisiones en base al dataset de validación es de: {(accuracy_score(target_val,pred_val))*100}')

La exactitud del árbol de decisiones en base al dataset de validación es de: 72.47278382581649


Vamos a hacer algunos ejercicios para determinar que hiperparámetros podemos modificar en el modelo para mejorar sus métricas. 

In [8]:
best_score = 0
for depth in range(1,31):
    model_tree_alt = DecisionTreeClassifier(random_state=123,max_depth=depth)
    model_tree_alt.fit(features_train,target_train)
    pred_val_alt = model_tree_alt.predict(features_val)
    actual_score = accuracy_score(target_val,pred_val_alt)*100
    if actual_score > best_score:
        print(f'Con una profundidad de {depth} la mejor exactitud del modelo es de {actual_score}')
        best_score = actual_score

Con una profundidad de 1 la mejor exactitud del modelo es de 73.56143079315707
Con una profundidad de 2 la mejor exactitud del modelo es de 76.51632970451011
Con una profundidad de 3 la mejor exactitud del modelo es de 78.38258164852256
Con una profundidad de 6 la mejor exactitud del modelo es de 78.84914463452566
Con una profundidad de 9 la mejor exactitud del modelo es de 79.47122861586314


La exactitud del modelo es mucho mejor definiendo la profundidad del árbol en 9.

In [9]:
model_tree_alt = DecisionTreeClassifier(random_state=123,max_depth=9,criterion='entropy')
model_tree_alt.fit(features_train,target_train)
pred_val_alt = model_tree_alt.predict(features_val)
actual_score = accuracy_score(target_val,pred_val_alt)*100
print(actual_score)

78.0715396578538


Es mejor dejar el hiperparámetro criterion en 'gini'

In [10]:
model_tree_alt = DecisionTreeClassifier(random_state=123,max_depth=9,splitter='random')
model_tree_alt.fit(features_train,target_train)
pred_val_alt = model_tree_alt.predict(features_val)
actual_score = accuracy_score(target_val,pred_val_alt)*100
print(actual_score)

77.60497667185071


Es mejor dejar el hiperparámetro splitter en 'best'

In [11]:
best_score = 0
for samples in range(2,21):
    model_tree_alt = DecisionTreeClassifier(random_state=123,max_depth=9,min_samples_split=samples)
    model_tree_alt.fit(features_train,target_train)
    pred_val_alt = model_tree_alt.predict(features_val)
    actual_score = accuracy_score(target_val,pred_val_alt)*100
    if actual_score > best_score:
        print(f'Con {samples} muestras, la exactitud del modelo es de {actual_score}')
        best_score = actual_score

Con 2 muestras, la exactitud del modelo es de 79.47122861586314
Con 11 muestras, la exactitud del modelo es de 79.62674961119751


Si bien, tener una mejora de 0.15% no parece un aumento significativo voy a usar el min_samples_split = 11.

In [12]:
best_score = 0
for samples in range(1,11):
    model_tree_alt = DecisionTreeClassifier(random_state=123,max_depth=9,min_samples_split=11,min_samples_leaf=samples)
    model_tree_alt.fit(features_train,target_train)
    pred_val_alt = model_tree_alt.predict(features_val)
    actual_score = accuracy_score(target_val,pred_val_alt)*100
    if actual_score > best_score:
        print(f'Con {samples} muestras, la exactitud del modelo es de {actual_score}')
        best_score = actual_score

Con 1 muestras, la exactitud del modelo es de 79.62674961119751


In [13]:
best_score = 0
for leaf in range(11,211,10):
    model_tree_alt = DecisionTreeClassifier(random_state=123,max_depth=9,min_samples_split=11,max_leaf_nodes=leaf)
    model_tree_alt.fit(features_train,target_train)
    pred_val_alt = model_tree_alt.predict(features_val)
    actual_score = accuracy_score(target_val,pred_val_alt)*100
    if actual_score > best_score:
        print(f'Con {leaf} muestras, la exactitud del modelo es de {actual_score}')
        best_score = actual_score

Con 11 muestras, la exactitud del modelo es de 78.69362363919129
Con 21 muestras, la exactitud del modelo es de 78.84914463452566
Con 31 muestras, la exactitud del modelo es de 79.16018662519441
Con 41 muestras, la exactitud del modelo es de 79.62674961119751
Con 51 muestras, la exactitud del modelo es de 79.93779160186625


Vamos a ajustar el hiperparámetro max_leaf_nodes = 51 ya que fue el valor que elevó la exactitud del modelo.

In [14]:
best_score = 0
list = [None,'sqrt','log2',2,0.75]
for each in list:
    model_tree_alt = DecisionTreeClassifier(random_state=123,max_depth=9,min_samples_split=11,max_leaf_nodes=51,max_features=each)
    model_tree_alt.fit(features_train,target_train)
    pred_val_alt = model_tree_alt.predict(features_val)
    actual_score = accuracy_score(target_val,pred_val_alt)*100
    if actual_score > best_score:
        print(f'Con el valor {each} en el hiperparámetro max_features, la exactitud del modelo es de {actual_score}')
        best_score = actual_score

Con el valor None en el hiperparámetro max_features, la exactitud del modelo es de 79.93779160186625


In [15]:
best_score = 0
list = [None,'balanced']
for each in list:
    model_tree_alt = DecisionTreeClassifier(random_state=123,max_depth=9,min_samples_split=11,max_leaf_nodes=51,class_weight=each)
    model_tree_alt.fit(features_train,target_train)
    pred_val_alt = model_tree_alt.predict(features_val)
    actual_score = accuracy_score(target_val,pred_val_alt)*100
    if actual_score > best_score:
        print(f'Con el valor {each} en el hiperparámetro class_weight, la exactitud del modelo es de {actual_score}')
        best_score = actual_score

Con el valor None en el hiperparámetro class_weight, la exactitud del modelo es de 79.93779160186625


#### 4.1.2 Evaluación del Modelo.

In [16]:
# Entrenamiento
model_tree_test = DecisionTreeClassifier(random_state=123,max_depth=9,min_samples_split=11,max_leaf_nodes=51)
model_tree_test.fit(features_train,target_train)

# Aplicación
pred_test = model_tree_test.predict(features_test)
print(f'La exactitud del modelo con el dataset de prueba es de: {accuracy_score(target_test,pred_test)*100}')

La exactitud del modelo con el dataset de prueba es de: 78.84914463452566


#### 4.2 Árboles Aleatorios.

Primero voy a entrenar el dataset de *features_train* con el algoritmo de aprendizaje *RandomForestClassifier* para obtener un modelo entrenado y después utilizar el *features_val* para validar las predicciones y ya después vealuar con diferentes métricas la calidad del modelo.

In [17]:
# Entrenamiento.
model_rt_tree = RandomForestClassifier(random_state=123)
model_rt_tree.fit(features_train,target_train)

# Aplicación.
pred_rt_val = model_rt_tree.predict(features_val)

##### 4.2.1 Métricas de evaluación.

Vamos a determinar la exactitud del modelo entrenado que generamos, comparando la variable *pred_val* contra *target_val*.

In [18]:
print(f'La exactitud del modelo árboles aleatorios en base al dataset de validación es de: {(accuracy_score(target_val,pred_rt_val))*100}')

La exactitud del modelo árboles aleatorios en base al dataset de validación es de: 80.09331259720062


Para hacer menos ejercicios individuales modificando hiperparámetros de manera individual para encontrar la mejor combinacioon, voy a utilizar GridSearchCV. 

In [19]:
# Voy a utilizar solo 2 ya que en un inicio use como 7 hiperparámetros y me resultaba en 38,000,000 de combinaciones
param_grid = {
    'n_estimators': [100,110,120,130,140,150],
    'max_depth' : [1,2,3,4,5,6,7,8,9,10],
}

rt = RandomForestClassifier(random_state=123)
grid_search = GridSearchCV(rt, param_grid=param_grid, cv=5, scoring='accuracy',verbose=1)
grid_search.fit(features_train, target_train)
print("Mejores hiperparámetros:", grid_search.best_params_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Mejores hiperparámetros: {'max_depth': 8, 'n_estimators': 130}


##### 4.2.2 Evaluación del Modelo.

In [20]:
# Entrenamiento
model_rt_tree_test = RandomForestClassifier(random_state=123,n_estimators=130,max_depth=8)
model_rt_tree_test.fit(features_train,target_train)

# Aplicación
pred_test_rt = model_rt_tree_test.predict(features_test)
print(f'La exactitud del modelo con el dataset de prueba es de: {accuracy_score(target_test,pred_test_rt)*100}')

La exactitud del modelo con el dataset de prueba es de: 81.18195956454122


#### 4.3 Gradient Boosting.

Primero voy a entrenar el dataset de *features_train* con el algoritmo de aprendizaje *GradientBoostingClassifier* para obtener un modelo entrenado y después utilizar el *features_val* para validar las predicciones y ya después vealuar con diferentes métricas la calidad del modelo.

In [21]:
# Entrenamiento.
model_boost = GradientBoostingClassifier(random_state=123)
model_boost.fit(features_train,target_train)

# Aplicación.
pred_boost_val = model_boost.predict(features_val)

##### 4.3.1 Métricas de evaluación.

Vamos a determinar la exactitud del modelo entrenado que generamos, comparando la variable *pred_val* contra *target_val*.

In [22]:
print(f'La exactitud del modelo gradient boosting en base al dataset de validación es de: {(accuracy_score(target_val,pred_boost_val))*100}')

La exactitud del modelo gradient boosting en base al dataset de validación es de: 78.84914463452566


Para hacer menos ejercicios individuales modificando hiperparámetros de manera individual para encontrar la mejor combinacioon, voy a utilizar GridSearchCV. 

In [23]:
param_grid = {
    'n_estimators': [100,110,120,130,140,150],
}

boost = GradientBoostingClassifier(random_state=123)
grid_search = GridSearchCV(boost, param_grid=param_grid, cv=5, scoring='accuracy',verbose=1)
grid_search.fit(features_train, target_train)
print("Mejores hiperparámetros:", grid_search.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Mejores hiperparámetros: {'n_estimators': 150}


##### 4.3.2 Evaluación del Modelo.

In [24]:
# Entrenamiento.
model_boost_test = GradientBoostingClassifier(random_state=123,n_estimators=150)
model_boost_test.fit(features_train,target_train)

# Aplicación.
pred_test_boost = model_boost_test.predict(features_test)
print(f'La exactitud del modelo gradient boosting en base al dataset de prueba es de: {(accuracy_score(target_test,pred_test_boost))*100}')

La exactitud del modelo gradient boosting en base al dataset de prueba es de: 80.87091757387248


#### 4.4 Regresión Logística.

Primero voy a entrenar el dataset de *features_train* con el algoritmo de aprendizaje *LogisticRegression* para obtener un modelo entrenado y después utilizar el *features_val* para validar las predicciones y ya después vealuar con diferentes métricas la calidad del modelo.

In [25]:
# Entrenamiento.
model_logic = LogisticRegression(random_state=123)
model_logic.fit(features_train,target_train)

# Aplicación.
pred_logic_val = model_logic.predict(features_val)

##### 4.4.1 Métricas de evaluación.

Vamos a determinar la exactitud del modelo entrenado que generamos, comparando la variable *pred_val* contra *target_val*.

In [26]:
print(f'La exactitud del modelo regresión logistica en base al dataset de validación es de: {(accuracy_score(target_val,pred_logic_val))*100}')

La exactitud del modelo regresión logistica en base al dataset de validación es de: 73.40590979782272


Para hacer menos ejercicios individuales modificando hiperparámetros de manera individual para encontrar la mejor combinacioon, voy a utilizar GridSearchCV. 

In [27]:
param_grid = {
    'penalty': ['l1','l2'],
    'solver' : ['liblinear'],
    'C' : [0.1,1,10,100],
}

linear = LogisticRegression(random_state=123)
grid_search = GridSearchCV(linear, param_grid=param_grid, cv=5, scoring='accuracy',verbose=1)
grid_search.fit(features_train, target_train)
print("Mejores hiperparámetros:", grid_search.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Mejores hiperparámetros: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


##### 4.4.2 Evaluación del Modelo.

In [28]:
# Entrenamiento.
model_logic_test = LogisticRegression(random_state=123,solver='liblinear',penalty='l1',C=10)
model_logic_test.fit(features_train,target_train)

# Aplicación.
pred_test_logic = model_logic_test.predict(features_test)
print(f'La exactitud del modelo Regresión Logística en base al dataset de prueba es de: {(accuracy_score(target_test,pred_test_logic))*100}')

La exactitud del modelo Regresión Logística en base al dataset de prueba es de: 74.96111975116641


### 5. Resumen.

La siguiente tabla muestra un resumen de los porcentajes obtenidos por diferentes modelos de ML. 

|      Modelo       |  Exactitud |
|-------------------|------------|
|Árbol de decisión  |    78.84   |
|Árboles aleatorios |    81.18   |
|     Gradiente     |    80.87   |
|Regresión Logistica|    74.96   |

### 6. Conclusiones.

En base a los datos obtenidos con los datasets de prueba para cada modelo de ML, podemos ver que el mejor modelo para evaluar los datos es el de Árboles Aleatorios, seguido del Gradient Boosting, Árbol de Decisión y por último Regresión Logística.

Si bien los Árboles Aleatorios y el Gradient Boosting son los mejores con una exactitud de mayor del 80%, llegan a consumir mucha memoria. Si recordamos se nos pide una exactitud mayor al 75% la cual practicamente cumplen también el árbol de decisión y Regresión Loigistica.

Con estos resultados se le puede presentar la información a la compañía Megaline y que ellos decidan si están dispuestos a usar los modelos con mayor exactitud y más uso de memoria o estám dispuestos a ceder ese 2-5% de exactitud por un modelo más rápido.
